In [6]:
'''
These are the necessary imports and downloads needed for the project to run
'''


!pip install transformers torch
!pip install git+https://github.com/PrithivirajDamodaran/ZSIC.git -q
!pip install gensim
!pip install nltk

import pickle
import pandas as pd
import numpy
import re
import os
import numpy as np
import collections
import operator
import itertools
import string
import csv
import transformers
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter
from sklearn.cluster import Birch
from sklearn import metrics
from PIL import Image
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

nltk.download('punkt')


/opt/conda/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
  Using cached transformers-4.18.0-py3-none-any.whl (4.0 MB)
  Using cached torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl (750.6 MB)
  Using cached sacremoses-0.0.49-py3-none-any.whl (895 kB)
  Using cached tokenizers-0.12.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (6.6 MB)
  Using cached regex-2022.3.15-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (749 kB)
  Using cached huggingface_hub-0.5.1-py3-none-any.whl (77 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 20.1
   

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

<h1> Outlier Filtration </h1>

In [7]:
'''
This is the dictionary that is going to store all the outliers with acceptable pictures
'''

all_outliers_sku = {}

<h3> Filter Outliers from original dataset </h3>

In [8]:
'''
This is the first step in finding all the outliers. An outlier is a SKU where the SKU has a segment name that is 
in the bottom 5% relative to the other segment names, has a category name that is in the bottom 5% relative to 
the other category names, has a class name that is in the bottom 5% relative to the other class names, 
and has a subclass name that is in the bottom 5% relative to the other subclass names.

seg_outliers is a list of the segment names that occurr 5% and below
category_outliers is a list of the category names that occurr 5% and below
cls_outliers is a list of the class names that occurr 5% and below
subcls_outliers is a list of the subclass names that occurr 5% and below

'''

df=pd.read_csv("/root/Capstone_Dataset_final.csv")
dataset = open('/root/image_info_final.txt', 'r')
relative_segments_counts = dict(df.groupby(["Segment Name"])["Segment Name"].count())
relative_category_counts = dict(df.groupby(["Category Name"])["Category Name"].count())
relative_class_counts = dict(df.groupby(["Class Name"])["Class Name"].count())
relative_subclass_counts = dict(df.groupby(["Sub-Class Name"])["Sub-Class Name"].count())
def find_outliers(vals,threshold):
    X = dict(sorted(vals.items(),key=lambda label:label[1]))
    outlier_amt = int(threshhold*len(X))
    outliers = list(X.keys())[0:outlier_amt]
    return outliers
threshhold = .05
seg_outliers = find_outliers(relative_segments_counts,threshhold)
category_outliers = find_outliers(relative_category_counts,threshhold)
cls_outliers = find_outliers(relative_class_counts,threshhold)
subcls_outliers = find_outliers(relative_subclass_counts,threshhold)

In [9]:
'''
Here we are gathering a list of all the SKU's that fall under a segment, category, class, 
or a subclass that is classified as an outlier


outlier_sku is a list of all the SKU's that are outliers
'''

df=pd.read_csv("/root/Capstone_Dataset_final.csv")
dataset = open('/root/image_info_final.txt', 'r')
outlier_sku = []
for i in seg_outliers:   
    for j in list(df.loc[df["Segment Name"]==i]["SKU"]):
        outlier_sku.append(j)
for i in category_outliers:   
    for j in list(df.loc[df["Category Name"]==i]["SKU"]):
        outlier_sku.append(j)
for i in cls_outliers:   
    for j in list(df.loc[df["Class Name"]==i]["SKU"]):
        outlier_sku.append(j)
for i in subcls_outliers:   
    for j in list(df.loc[df["Sub-Class Name"]==i]["SKU"]):
        outlier_sku.append(j)
outlier_sku = list(set(outlier_sku))
len(outlier_sku)

244

<h3>Find outliers with images</h3>

In [10]:
'''
Here we are mapping all of the SKU's to the SKU's accociated pictures, if it has one. In addition we go 
through all of the pictures and do a check if they are in an acceptable format.

all_outliers_sku is a dictionary of the outlier SKU's that have accpetable pictures

'''

df=pd.read_csv("/root/Capstone_Dataset_final.csv")
dataset = open('/root/image_info_final.txt', 'r')
SKUToProduct_seg = df.groupby('SKU')['Segment Name'].apply(list)
SKUToProduct_cat = df.groupby('SKU')['Category Name'].apply(list)
SKUToProduct_cls = df.groupby('SKU')['Class Name'].apply(list)
SKUToProduct_subcls = df.groupby('SKU')['Sub-Class Name'].apply(list)
segToImage_specific = collections.defaultdict(list)
catToImage_specific = collections.defaultdict(list)
clsToImage_specific = collections.defaultdict(list)
subclsToImage_specific = collections.defaultdict(list)
productDict_cat = collections.defaultdict(dict)
productDict_seg = collections.defaultdict(dict)
productDict_cls = collections.defaultdict(dict)
productDict_subcls = collections.defaultdict(dict)

pictures = []
lines = dataset.readlines()[1:]
i = 0
for line in lines:
    x = line.split(" | ")
    pictures.append(x)

for picture in pictures:
    sku = int(picture[0])
    images = picture[1].lstrip().split(" ")
    
    productName_seg = SKUToProduct_seg[sku][0]
    productName_cat = SKUToProduct_cat[sku][0]
    productName_cls = SKUToProduct_cls[sku][0]
    productName_subcls = SKUToProduct_subcls[sku][0]
    for img in images:
        segToImage_specific[sku].append(img)
        catToImage_specific[sku].append(img)
        clsToImage_specific[sku].append(img)
        subclsToImage_specific[sku].append(img)
        
    productDict_seg[productName_seg][sku] = segToImage_specific[sku]
    productDict_cat[productName_cat][sku] = catToImage_specific[sku]
    productDict_cls[productName_cls][sku] = clsToImage_specific[sku]
    productDict_subcls[productName_subcls][sku] = subclsToImage_specific[sku]
for i in seg_outliers:   
    for j in productDict_seg[i].items():
        all_outliers_sku[j[0]] = j[1]
for i in category_outliers:   
    for j in productDict_cat[i].items():
        if j[0] not in all_outliers_sku:
            all_outliers_sku[j[0]] = j[1]
for i in cls_outliers:   
    for j in productDict_cls[i].items():
        if j[0] not in all_outliers_sku:
            all_outliers_sku[j[0]] = j[1]
for i in subcls_outliers:   
    for j in productDict_subcls[i].items():
        if j[0] not in all_outliers_sku:
            all_outliers_sku[j[0]] = j[1]

non_acceptable_pictures = []

acceptable_format = ["jp2","jpeg","png","jp2\n","jpeg\n","png\n"]
for i in all_outliers_sku.items():
    for j in i[1]:
        test = j.split(".")
        if test[1] not in acceptable_format:
            non_acceptable_pictures.append(i[0])
            break

for i in non_acceptable_pictures:
    all_outliers_sku.pop(i)

In [11]:
'''
Here we are querying the original dataframe to get all of the outliers SKU's without pictures or outlier 
SKU's that do not have pictures in accpetable formats.

df_outliers_no_pic is a dataframe of outliers with no pictures or outliers that have an unacceptable picture format
'''


df_outliers = df[df["SKU"].isin(outlier_sku)]
df_outliers_no_pic = df_outliers[~(df_outliers["SKU"].isin(all_outliers_sku.keys()))]

In [12]:
len(df[~df["SKU"].isin(outlier_sku)])

29756

<h3> Outliers without images </h3>

In [13]:
'''
Here we are creating a dictionary for the outliers without a picture or outliers with an unnaceptable picture.
The first choice is the SKU's Item Description, the second choice is the SKU's Item Description 2, the third choice is the SKU's Item Title,
and the last choice is the SKU's Manufacturer name. If all 4 of those fields are null for a SKU then we add it to a seperate list.

product_Dict_text is a dictionary that mapes SKU's to either its Item Description, Item Description 2, ItemTitle, or its manufacturer.
no_des is a list of SKU's that do not have any pictures, acceptable pictures, and no acceotable text desciprtion
'''


SKUtoProduct_seg = df_outliers_no_pic.groupby("SKU")["Segment Name"].apply(list)
SKUtoProduct_cat = df_outliers_no_pic.groupby("SKU")["Category Name"].apply(list)
SKUtoProduct_cls = df_outliers_no_pic.groupby("SKU")["Class Name"].apply(list)
SKUtoProduct_subcls = df_outliers_no_pic.groupby("SKU")["Sub-Class Name"].apply(list)

SKUToDescription1 = df_outliers_no_pic.groupby("SKU")["ItemDescription"].apply(list)
SKUToDescription2 = df_outliers_no_pic.groupby("SKU")["ItemDescription_2"].apply(list)
SKUToDescription3 = df_outliers_no_pic.groupby("SKU")["ItemTitle"].apply(list)
SKUToDescription4 = df_outliers_no_pic.groupby("SKU")["Manufacturer"].apply(list)

product_Dict_text = {}

SKU = list(df_outliers_no_pic["SKU"])
# print(SKU)
# for i in non_acceptable_pictures:
#     SKU.append(i)
# print(SKU)
no_des = []
for sku in SKU:
    productName_seg = SKUtoProduct_seg[sku][0]
    productName_cat = SKUtoProduct_cat[sku][0]
    productName_cls = SKUtoProduct_cls[sku][0]
    productName_subcls = SKUtoProduct_subcls[sku][0]
    
    des1,des2,des3,des4 = SKUToDescription1.get(sku,[0.0])[0],SKUToDescription2.get(sku,[0.0])[0],SKUToDescription3.get(sku,[0.0])[0],SKUToDescription4.get(sku,[0.0])[0]
    if type(des1)==type("string"):
        product_Dict_text[sku] = des1
    elif type(des2)==type("string"):
        product_Dict_text[sku] = des2
    elif type(des3)==type("string"):
        product_Dict_text[sku] = des3
    elif type(des4)==type("string"):
        product_Dict_text[sku] = des4
    else:
        no_des.append(sku)
len(product_Dict_text.keys())

21

<h3> Initialize Text Model </h3>

In [387]:
'''
Here we are initializing the text based zero shot model
'''


classifier = transformers.pipeline("zero-shot-classification",model="facebook/bart-large-mnli")

In [17]:
len(product_Dict_text)
len(all_outliers_sku)

223

<h3>Text Zero Shot Model</h3>

In [ ]:
'''
Here we are using the text based zero shot model to predict the SKU's segment names, 
category names, class names, and subclass names based on the text descrtiption that the SKU is mapped to.
We feed in all the unique segment names as the possible choices for the SKU's segment classification.
We feed in all the unique category names as the possible choices for the SKU's category classification.
We feed in all the unique class names as the possible choices for the SKU's class classification.
We feed in all the unique subclass names as the possible choices for the SKU's subclass classification.

text_outliers is a mapping of an SKU to a list of classifications/classifications confidence scores.
Here is the format of a key-value pair in text outliers:
SKU -> [[subclass name, subclass confidence],[class name, class confidence],[category name, category confidence],[segment name, segment confidence]]
'''
text_outliers = collections.defaultdict(list)

labels_seg = list(df["Segment Name"].unique())
labels_cat = list(df["Category Name"].unique())
labels_cls = list(df["Class Name"].unique())
labels_subcls = list(df["Sub-Class Name"].unique())
count = 0
for sku,des in product_Dict_text.items():
    count+=1
    print("SKU {} out of {}".format(count,len(product_Dict_text)))
    preds_seg = classifier(des, labels_seg, multi_label=True)
    preds_cat = classifier(des, labels_cat, multi_label=True)
    preds_cls = classifier(des, labels_cls, multi_label=True)
    preds_subcls = classifier(des, labels_subcls, multi_label=True)
    text_outliers[sku].append([preds_subcls['labels'][0],preds_subcls['scores'][0]])
    text_outliers[sku].append([preds_cls['labels'][0],preds_cls['scores'][0]])
    text_outliers[sku].append([preds_cat['labels'][0],preds_cat['scores'][0]])
    text_outliers[sku].append([preds_seg['labels'][0],preds_seg['scores'][0]])
    
    
    
text_outliers

SKU 1 out of 21
SKU 2 out of 21
SKU 3 out of 21
SKU 4 out of 21
SKU 5 out of 21
SKU 6 out of 21
SKU 7 out of 21
SKU 8 out of 21
SKU 9 out of 21
SKU 10 out of 21


<h3>Zero Shot Image Model </h3>

In [ ]:
'''
Here we are initializing the image based zero shot model.
'''

import torch
from ZSIC import ZeroShotImageClassification

zsic = ZeroShotImageClassification()

In [ ]:
'''
Here we are using the image based zero shot model to predict the SKU's segment names, 
category names, class names, and subclass names based on the image or images that a SKU is a mapped too.
If an SKU has multiple images then the image that produces the higest confidence score for a particular classification is used.

We feed in all the unique segment names as the possible choices for the SKU's segment classification.
We feed in all the unique category names as the possible choices for the SKU's category classification.
We feed in all the unique class names as the possible choices for the SKU's class classification.
We feed in all the unique subclass names as the possible choices for the SKU's subclass classification.

accurate_segments is a dictionary of the SKU's where a SKU is mapped to the segment classification and segment classification confidence score
accurate_category is a dictionary of the SKU's where a SKU is mapped to the cateogory classification and cateogry classification confidence score
accurate_class is a dictionary of the SKU's where a SKU is mapped to the class classification and class classification confidence score
accurate_subclass is a dictionary of the SKU's where a SKU is mapped to the subclass classification and subclass classification confidence score
'''


inp_image = "/root/Capstone_images_set_final/"
labels_seg = list(df["Segment Name"].unique())
labels_cat = list(df["Category Name"].unique())
labels_cls = list(df["Class Name"].unique())
labels_subcls = list(df["Sub-Class Name"].unique())

correct = 0
accurate_segments = collections.defaultdict(list)
accurate_category = collections.defaultdict(list)
accurate_class = collections.defaultdict(list)
accurate_subclass = collections.defaultdict(list)
count = 0
for sku,picture in all_outliers_sku.items():
    count+=1
    print("SKU {} out of {}".format(count,len(all_outliers_sku)))
    for j in picture:
        j = j.replace("\n","")
        j = Image.open(inp_image + j)

        preds_seg = zsic(image=j,
                    candidate_labels=labels_seg) 
        preds_cat = zsic(image=j,
                    candidate_labels=labels_cat)
        preds_cls = zsic(image=j,
                    candidate_labels=labels_cls)
        preds_subcls = zsic(image=j,
                    candidate_labels=labels_subcls)
        if sku not in accurate_segments:
            accurate_segments[sku].append(preds_seg['labels'][0])
            accurate_segments[sku].append(preds_seg['scores'][0])
        else:
            if preds_seg['scores'][0]>accurate_segments[sku][1]:
                accurate_segments[sku][0] = preds_seg['labels'][0]
                accurate_segments[sku][1] = preds_seg['scores'][0]
    
        if sku not in accurate_category:
            accurate_category[sku].append(preds_cat['labels'][0])
            accurate_category[sku].append(preds_cat['scores'][0])
        else:
            if preds_cat['scores'][0]>accurate_category[sku][1]:
                accurate_category[sku][0] = preds_cat['labels'][0]
                accurate_category[sku][1] = preds_cat['scores'][0]
                
        if sku not in accurate_class:
            accurate_class[sku].append(preds_cls['labels'][0])
            accurate_class[sku].append(preds_cls['scores'][0])
        else:
            if preds_cls['scores'][0]>accurate_class[sku][1]:
                accurate_class[sku][0] = preds_cls['labels'][0]
                accurate_class[sku][1] = preds_cls['scores'][0]
                
        if sku not in accurate_subclass:
            accurate_subclass[sku].append(preds_subcls['labels'][0])
            accurate_subclass[sku].append(preds_subcls['scores'][0])
        else:
            if preds_subcls['scores'][0]>accurate_subclass[sku][1]:
                accurate_subclass[sku][0] = preds_subcls['labels'][0]
                accurate_subclass[sku][1] = preds_subcls['scores'][0]

In [ ]:
'''
Here we are orgainizing all the SKU's and their classifications along with their classification confidence scores.
This includes all the SKU's that were processed with the image based zero shot model, and text based zero shot model.
If an outlier SKU did not have an acceptable image or an appropriate text input then we go with the classification on 
the dataset with 100% confidence.

outlier_accuracy is a mapping of an SKU to a list of classifications/classifications confidence scores.
Here is the format of a key-value pair in outlier_accuracy:
SKU -> [[subclass name, subclass confidence],[class name, class confidence],[category name, category confidence],[segment name, segment confidence]]

'''
outlier_accuracy = collections.defaultdict(list)
for i in accurate_segments.items():
    outlier_accuracy[i[0]].append(accurate_subclass[i[0]])
    outlier_accuracy[i[0]].append(accurate_class[i[0]])
    outlier_accuracy[i[0]].append(accurate_category[i[0]])
    outlier_accuracy[i[0]].append(i[1])
    

for i in no_des:
    outlier_accuracy[i].append([list(df_outliers_no_pic.loc[test["SKU"]==i]["Sub-Class Name"]),1.0]) 
    outlier_accuracy[i].append([list(df_outliers_no_pic.loc[test["SKU"]==i]["Class Name"]),1.0]) 
    outlier_accuracy[i].append([list(df_outliers_no_pic.loc[test["SKU"]==i]["Category Name"]),1.0]) 
    outlier_accuracy[i].append([list(df_outliers_no_pic.loc[test["SKU"]==i]["Segment Name"]),1.0]) 
    
    
for i in text_outliers.items():
    outlier_accuracy[i[0]]=i[1]

<h1> Unsupervised Learning Model </h1>

<h3> Read Data </h3>

In [306]:
df_arvind=pd.read_csv("/root/Capstone_Dataset_final.csv")
data  = df_arvind[~(df_arvind["SKU"].isin(outlier_sku))]
data = data.reset_index(drop=True)
data

,SKU,ItemTitle,ItemDescription,ItemBulletPoint,ItemDescription_2,Manufacturer,MfrPartNum,SellUOM,ItemPrice,ItemFactTag,Segment,Segment Name,Category,Category Name,Class,Class Name,Sub-Class,Sub-Class Name
0,131645,Gibson Studio California Mauna 12-Piece Melami...,"Whether you're dining indoors or outdoors, thi...",<ul><li>Melamine construction for rugged durab...,NaN,GIBSON OVERSEAS INC.,995105704M,EA,31.41,NaN,SEG_11000000,Food & Beverage,CAT_11500000,Food & Beverage Serving,CLS_11540000,Tableware,SUB_11540008,Sets - Dinnerware
1,42483,Thronmax X1 Webcam - 30 fps - USB 2.0 - 1 Pack...,NaN,<ul><li>Video conferencing couldn't get any be...,<p><b>Suitable for Most Software </b></p> <p>P...,THRONMAX,X1,EA,NaN,NaN,SEG_400000,Computers & Hardware,CAT_420000,Computer Accessories,CLS_424000,"Calibrators, Webcams & Hubs",SUB_424003,Webcams
2,175678,IOGEAR Thunderbolt 3 USB-C 2m 20Gbps Cable - 6...,NaN,<ul><li>Power delivery up to 100W (20V @5A) us...,<p><b>Thunderbolt™ 3 - Premium Performance for...,ATEN TECHNOLOGIES,GT3C02,EA,NaN,NaN,SEG_20000000,Cords & Cables,CAT_20500000,Power & Data,CLS_20510000,Cables - Data Transfer,SUB_20510002,Cables - Data
3,190527,Hoffman Tech 677-12E-HTI Remanufactured Extra-...,Remanufactured from an original HP recycled ca...,<ul><li>Comparable to the HP 12A (Q2612D) cart...,NaN,"IMAGE PROJECTIONS WEST, INC.",677-12E-HTI,EA,62.94,"* Quality toner at a savings * Yields up to 4,...",SEG_24000000,"Ink, Toner, Ribbons & 3D Filaments",CAT_24400000,Toner,CLS_24410000,Machine Toner,SUB_5070132,"Cartridges - Toner, Color"
4,145583,Suncast Commercial Paper Recycling Lid For 23-...,Keep recyclables (where facilities exist) cont...,<ul><li>Made of durable resin.</li> <li>Fits m...,NaN,SUNCAST CORPORATION,TCNLID03BLD,EA,NaN,NaN,SEG_14000000,Janitorial,CAT_14600000,Trash Handling,CLS_14620000,Trash Receptacles,SUB_14620007,Trash Can Lids
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29751,234370,HPE Care Pack Proactive Care Service with Defe...,NaN,<ul><li>Protects your investment and adds valu...,HPE Care Pack Support Plus Service provides co...,HP INC.,U6DV8E,EA,NaN,NaN,SEG_33000000,Services,CAT_33500000,Services - Tech,CLS_33530000,Services - IT,SUB_33530006,Managed Service Plans
29752,63119,"VELCRO® Brand Tape Roll, Hook, 2"" x 75', Black","Securely hang posters, decorations, tools and ...",<ul><li>Safe for use on various surfaces.</li>...,NaN,"B O X MANAGEMENT, INC.",VEL137,CA,NaN,NaN,SEG_900000,"Writing, Art & Drafting",CAT_950000,Pens,CLS_951000,Pens - Ink,SUB_951001,Pens - Ballpoint
29753,27139,"Blue Sky™ Academic Weekly/Monthly Planner, Let...",Keep your events organized with the Blue Sky A...,<ul><li>2-page weekly and monthly spreads prov...,NaN,BLUE SKY THE COLOR OF IMAGINATION LLC,100135-A23,EA,NaN,NaN,SEG_6000000,Mailing & Shipping,CAT_6600000,Shipping Supplies,CLS_6610000,Protective Supplies,SUB_6610005,Poly Sheeting
29754,126875,"Cambro Deli Crocks, 2.7 Qt, Black, Pack Of 6 C...",Make setting up your spread of prepped items s...,<ul><li>Made from high-impact plastic (FDA app...,NaN,CAMBRO MFG. CO.,CAMCP27110,CA,NaN,NaN,SEG_20000000,Cords & Cables,CAT_20300000,Cables - Computer,CLS_20310000,Cables - Computer Network,SUB_20310001,Cables - Ethernet


<h3> Create Tokens for Product description </h3>

In [307]:
def tokenize_dataset(data):
    language = "en"
    new_data = []
    regex = re.compile('<.*?>')
    count = 1
    segments = []
    categories = []
    classes = []
    sub_classes = []
    for index, row in data.iterrows():
        sku = row["SKU"]
        manu = row["Manufacturer"]
        desc = row["ItemDescription"]
        desc_2 = row["ItemDescription_2"]
        title = row["ItemTitle"]
        bullet = row["ItemBulletPoint"]
        segment = row["Segment"]
        category = row["Category"]
        class_ = row["Class"]
        sub_class = row["Sub-Class"]
        
        segments.append(segment)
        categories.append(category)
        classes.append(class_)
        sub_classes.append(sub_class)
        
        words = []
        keywords = []
        keywords_2 = []
        keywords_3 = []
        total_text = ""
        if(type(desc) is str):
            total_text += desc        
        total_text = total_text.lower()
        total_text = re.sub(r"\[(.*?)\]", "", total_text)  # Remove [+XYZ chars] in content
        total_text = re.sub(r"\s+", " ", total_text)  # Remove multiple spaces in content
        total_text = re.sub(r"\w+…|…", "", total_text)  # Remove ellipsis (and last word)
        total_text = re.sub(r"(?<=\w)-(?=\w)", " ", total_text)  # Replace dash between words
        total_text = re.sub(
            f"[{re.escape(string.punctuation)}]", "", total_text
        )  # Remove punctuation
        
        tokens = word_tokenize(total_text)
        output = str(count) + "/" + str(len(data))
        count +=1
        print(output, end = "\r")
        new_data.append(tokens)
    return new_data, segments, categories, classes, sub_classes

In [308]:
new_data, train_segments, train_categories, train_classes, train_sub_classes = tokenize_dataset(data)

<h3> Vectorize </h3>

In [309]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(new_data)]
model = Doc2Vec(documents, window=2, min_count=1, workers=4)

In [310]:
start_alpha=0.01
X = []
count = 1
for tokens in new_data:
    X.append( model.infer_vector(tokens, alpha=start_alpha))
    output = str(count) + "/" + str(len(new_data))
    print(output, end = "\r")
    count+=1

<h3>Find Cluster Sizes</h3>

In [311]:
unique_segments = len(list(set(train_segments)))
print("Unique Segments:", unique_segments)

unique_categories = len(list(set(train_categories)))
print("Unique Categories:", unique_categories)

unique_classes = len(list(set(train_classes)))
print("Unique Classes:", unique_classes)

unique_sub_classes = len(list(set(train_sub_classes)))
print("Unique Sub Classes:", unique_sub_classes)

Unique Segments: 39
Unique Categories: 220
Unique Classes: 611
Unique Sub Classes: 1656


<h3> Train Models </h3>

In [312]:
segment_model = Birch(branching_factor=50, n_clusters=unique_segments, threshold=.2, compute_labels=True)
segment_model.fit(X)
segment_labels = segment_model.labels_
print(f"Silhouette coefficient: {metrics.silhouette_score(X, segment_model.labels_):0.2f}")

category_model = Birch(branching_factor=50, n_clusters=unique_categories, threshold=0.2, compute_labels=True)
category_model.fit(X)
category_labels = category_model.labels_
print(f"Silhouette coefficient: {metrics.silhouette_score(X, category_model.labels_):0.2f}")

class_model = Birch(branching_factor=50, n_clusters=unique_classes, threshold=0.2, compute_labels=True)
class_model.fit(X)
class_labels = class_model.labels_
print(f"Silhouette coefficient: {metrics.silhouette_score(X, class_model.labels_):0.2f}")

sub_class_model = Birch(branching_factor=50, n_clusters=unique_sub_classes, threshold=0.2, compute_labels=True)
sub_class_model.fit(X)
sub_class_labels = sub_class_model.labels_
print(f"Silhouette coefficient: {metrics.silhouette_score(X, sub_class_model.labels_):0.2f}")

Silhouette coefficient: 0.27
Silhouette coefficient: 0.28
Silhouette coefficient: 0.30
Silhouette coefficient: 0.32


<h3> Find Cluster Counts and Mappings for Labels </h3>

In [313]:
def create_mapping(labels, train_labels):
    unique_elements = list(set(train_labels))

    cluster_counts = {}
    mapping = {}
    for x in range (0, len(unique_elements)):
        cluster_counts[x] = list()

        mapping[x] = list()

    for x in range(len(train_labels)):
        cluster_counts[labels[x]].append(train_labels[x])

    for x in range(0, len(unique_elements)):
        c = Counter(cluster_counts[x])
        if(len(c) != 0):
            most_common = c.most_common()
            mapping[x] = most_common     
    return mapping

In [314]:
def get_level_count(labels):
    count = {}
    unique_labels = list(set(labels))
    for x in unique_labels:
        count[x] = 0
        
    c = Counter(labels)
    for x in c.most_common():
        count[x[0]] = x[1]
    return count

In [315]:
segment_range = 2
category_range = 2
class_range = 2
sub_class_range = 2

In [316]:
segment_map = create_mapping(segment_labels, train_segments)
category_map = create_mapping(category_labels, train_categories)
class_map = create_mapping(class_labels, train_classes)
sub_class_map = create_mapping(sub_class_labels, train_sub_classes)

In [317]:
segment_count = get_level_count(train_segments)
category_count = get_level_count(train_categories)
class_count = get_level_count(train_classes)
sub_class_count = get_level_count(train_sub_classes)

In [318]:
segment_predictions = segment_labels
category_predictions = category_labels
class_predictions = class_labels
sub_class_predictions = sub_class_labels

<h3> Calculate Average Precision and Accuracy </h3>

In [319]:
segment_accuracy = 0
category_accuracy = 0
class_accuracy = 0
sub_class_accuracy = 0
precision = 0

for x in range(len(segment_predictions)):
    segment = train_segments[x]
    category = train_categories[x]
    class_ = train_classes[x]
    sub_class = train_sub_classes[x]
    current_precision = 0
    segments = list(map(operator.itemgetter(0), segment_map[segment_predictions[x]][0:segment_range]))
    if(segment in segments):
        segment_accuracy +=1
        current_precision += .25

    categories = list(map(operator.itemgetter(0), category_map[category_predictions[x]][0:category_range]))
    if(category in categories):
        category_accuracy +=1
        if(current_precision == .25):
            current_precision += .25
        
    classes = list(map(operator.itemgetter(0), class_map[class_predictions[x]][0:class_range]))
    if(class_ in classes):
        class_accuracy +=1
        if(current_precision == .5):
            current_precision += .25
    
    sub_classes = list(map(operator.itemgetter(0), sub_class_map[sub_class_predictions[x]][0:sub_class_range]))
    if(sub_class in sub_classes):
        sub_class_accuracy +=1
        if(current_precision == .75):
            current_precision += .25
    precision += current_precision
    

segment_accuracy = segment_accuracy/len(segment_predictions)
category_accuracy = category_accuracy/len(category_predictions)
class_accuracy = class_accuracy/len(class_predictions)
sub_class_accuracy = sub_class_accuracy/len(sub_class_predictions)
precision = precision/len(segment_predictions)


print("Results - Average over whole dataset")
print()
print(f"Segment Accuracy: {segment_accuracy:0.2f}")
print(f"Category Accuracy: {category_accuracy:0.2f}")
print(f"Class Accuracy: {class_accuracy:0.2f}")
print(f"Sub Class Accuracy: {sub_class_accuracy:0.2f}")
print(f"Precision: {precision:0.2f}")

Results - Average over whole dataset

Segment Accuracy: 0.34
Category Accuracy: 0.28
Class Accuracy: 0.27
Sub Class Accuracy: 0.30
Precision: 0.20


<h3> Calculate Confidence Scores and Output to CSV File </h3>

In [320]:
def get_confidence_score(mapping, prediction, x_val, counts):
    confidence = 0
    
    if(not pd.isna(x_val)):
        total = counts[x_val]
        index = list(map(operator.itemgetter(0), mapping[prediction])).index(x_val)
        cluster_count = mapping[prediction][index][1]
        confidence = cluster_count/total

        
    
    return confidence

In [331]:
def output_results(data, header, segment_predictions, category_predictions, class_predictions, 
                   sub_class_predictions, segment_map, category_map, class_map, sub_class_map,
                  segment_count, category_count, class_count, sub_class_count):   
    results = []
    count = 1
    for index, row in data.iterrows():
        sku = row["SKU"]
        title = row["ItemTitle"]
        segment = row["Segment"]
        category = row["Category"]
        class_ = row["Class"]
        sub_class = row["Sub-Class"]
        segment_name = row["Segment Name"]
        category_name = row["Category Name"]
        class_name = row["Class Name"]
        sub_class_name = row["Sub-Class Name"]


        segment_prediction = segment_predictions[index]
        segment_confidence = get_confidence_score(segment_map, segment_prediction, segment, segment_count)

        category_prediction = category_predictions[index]
        category_confidence = get_confidence_score(category_map, category_prediction, category, category_count)

        class_prediction = class_predictions[index]
        class_confidence = get_confidence_score(class_map, class_prediction, class_, class_count)
        

        sub_class_prediction = sub_class_predictions[index]
        sub_class_confidence = get_confidence_score(sub_class_map, sub_class_prediction, sub_class, sub_class_count)


        result = [sku, title, sub_class_name, class_name, category_name, segment_name, 
                  sub_class_confidence, class_confidence, category_confidence, segment_confidence]
        results.append(result)


        output = str(count) + "/" + str(len(data))
        count +=1
        print(output, end = "\r")
    for i in outlier_accuracy.items():
        # Appending result from outlier accuracies that were calculated above
        results.append([i[0],list(df_arvind.loc[df_arvind["SKU"]==i[0]]["ItemTitle"])[0],i[1][0][0],i[1][1][0],i[1][2][0],i[1][3][0],i[1][0][1],i[1][1][1],i[1][2][1],i[1][3][1]])
    with open('results.csv', 'w+', encoding='UTF8', newline='') as f:
        writer = csv.writer(f)

        # write the header
        writer.writerow(header)

        # write multiple rows
        writer.writerows(results)

    print("Finished: Wrote to CSV file results.csv")

In [332]:
header = ['SKU', 'ItemTitle', 'Sub-Class Name', 'Class Name', 'Category Name', 'Segment Name', 
          'Sub-Class Score', 'Class Score', 'Category Score', 'Segment Score']
output_results(data, header, segment_predictions, category_predictions, class_predictions, 
               sub_class_predictions, segment_map, category_map, 
               class_map, sub_class_map, segment_count, category_count, class_count,
              sub_class_count)

Finished: Wrote to CSV file results.csv
